In [26]:
import os
import numpy as np
from skimage import io


import tensorflow
from tensorflow import keras
import pandas as pd

from PIL import Image, ImageOps


In [27]:
model = keras.models.load_model('new2.h5')

In [31]:
logo=Image.open(r"C:\Users\Sanket M\Desktop\Untitled.png")
logo=ImageOps.grayscale(logo)
logo=np.asarray(logo)
coords=[]
xycoords=[]
def black_and_white(a):# takes np array image
    m=a.copy()
    for i in range(len(m)):
        for j in range(len(m[0])):
            if m[i][j] >200:
                m[i][j]=255
            else:
                m[i][j]=0
    return m
def line_coords(coords):
    xmin=coords[0][0][0]
    xmax=coords[-1][0][0]
    ymin=20000
    ymax=0
    for i in coords:
        for j in i:
            if j[1] >ymax:
                ymax=j[1]
            if j[1] < ymin:
                ymin=j[1]
    xycoords.append([xmin,xmax+2,ymin+1,ymax])
for i in range(len(logo[1:-1,1:-1])):
    coo=[]
    flag=0
    for c in logo[1:-1,1:-1][i]:
        if c<200:
            flag=1
    if flag==1:
        for b in range(len(logo[1:-1,1:-1][i])):
            if logo[1:-1,1:-1][i][b]>200:
                try:
                    if logo[1:-1,1:-1][i][b+1] <200:
                        coo.append([i,b+1])
                        #print(i,b+1)
                except:
                    pass
            if logo[1:-1,1:-1][i][b]<200:
                try:
                    if logo[1:-1,1:-1][i][b+1] > 200:
                        coo.append([i,b])
                        #print(i,b+1)

                except:
                    pass
    else:
        if len(coords)>0:
            line_coords(coords)
            coords=[]
        
    if len(coo)>0:
        #print('hurrah')
        coords.append(coo)
spaces=np.array([])
sp=[]
ctr=0
for x in xycoords:
    a=black_and_white(logo[x[0]:x[1],x[2]:x[3]])
    for i in range(len(a[0,:])):

        f=0
        for j in a[:,i]:
            if j==0:
                f=1
        if f!=1:
            ctr+=1
        if f==1:
            sp.append(ctr)
            ctr=0
    import ckwrap
    nums= np.array([i for i in sp if i!=0])
    km = ckwrap.ckmeans(nums,2)
    spaces=np.concatenate((spaces,np.array([0]),km.labels), axis=None)
count=0
for hoe in xycoords:    
    newC=[0]
    def flagCalc(i):
        flag = 1
        jo=0
        for j in range(len(logo[hoe[0]:hoe[1],hoe[2]:hoe[3]][:,i])):
            if  logo[hoe[0]:hoe[1],hoe[2]:hoe[3]][:,i][j]<150:
                flag=0
                jo=j
        #print(flag)
        return flag        

    for i in range(len(logo[hoe[0]:hoe[1],hoe[2]:hoe[3]][0,:])):
        try:
            if flagCalc(i)<flagCalc(i+1):
                newC.append(i+1)
        except:
            pass
    newC.append(hoe[3])
    for i in range(len(newC)-1):   
        A=black_and_white(logo[hoe[0]:hoe[1],hoe[2]:hoe[3]][:,newC[i]:newC[i+1]])
        im = Image.fromarray(A)
        #im.save("dump/"+str(count)+".png")
        count+=1  

In [33]:
spaces

array([0., 0., 0., 0., 0., 1., 0., 0.])

# supporting functions

In [34]:
def borderRemoval(path):
    a = io.imread(path)
   
    #print(a)
    for i in range(len(a)):
        for j in range(len(a[0])):
            if a[i][j] >200:
                a[i][j]=255
            else:
                a[i][j]=0
    def flagCalc(i):
        flag = 0
        for j in range(len(i)):
            if  i[j]==0:
                flag=1

        return flag       
    y1=0
    y2=a.shape[0]
    x1=0
    x2=a.shape[1]
    
    
    for i in range(len(a)-1):
        #print(flagCalc(a[i])) 
        if flagCalc(a[i])<flagCalc(a[i+1]):
            y2=a.shape[0]
            if (i+1)< y2:
                y1=i+1
        elif flagCalc(a[i])>flagCalc(a[i+1]):
            if (i-1)>y1:
                y2=i-1
    for i in range(len(a[0,:])-1):
        #print(flagCalc(a[i])) 
        if flagCalc(a[:,i])<flagCalc(a[:,i+1]):
            if (i+1)< x2:
                x1=i+1
        elif flagCalc(a[:,i])>flagCalc(a[:,i+1]):
            if (i-1)>x1:
                x2=i-1   
    im = Image.fromarray(a[y1:y2,x1:x2])
    print(y1,y2,x1,x2)
    im.save(path)
def shahidAlgo(path):
    a = io.imread(path)
    if a.shape[0] > a.shape[1]  :
        f=28/a.shape[0]
    else:
        f=28/a.shape[1]
    b=Image.fromarray(a,mode='L').resize((int(a.shape[1]*f),int(a.shape[0]*f)),Image.BICUBIC)
    c=Image.fromarray(np.full((32, 32), 255).astype('uint8'),mode='L')
    img_w, img_h = b.size
    bg_w, bg_h = c.size
    offset = ((bg_w - img_w) // 2, (bg_h - img_h) // 2)
    c.paste(b, offset)
    
    c.save(path)
def BnW(path):
    a = io.imread(path)
    for i in range(len(a)):
        for j in range(len(a[0])):
            if a[i][j]>200:
                a[i][j] = 255
            else:
                a[i][j]=0
    Image.fromarray(a,mode='L').save(path)


In [12]:
for mm in os.listdir(r"dump/"):
    #print(r"dump/"+str(mm))
    borderRemoval(r"dump/"+str(mm))
    shahidAlgo(r"dump/"+str(mm))
    BnW(r"dump/"+str(mm))

1 12 0 26
6 86 15 80
29 85 4 110
29 86 4 82
29 85 7 79
5 86 40 108
5 86 10 75
29 85 4 108


In [13]:
labs={0: 0,1: 1, 2: 2, 3: 3,4: 4,5: 5,6: 6,7: 7,8: 8,9: 9,
 10: 'A',
 11: 'B', 12: 'C', 13: 'D', 14: 'E',
 15: 'F', 16: 'G', 17: 'H',
 18: 'I', 19: 'J',20: 'K',
 21: 'L',22: 'M',
 23: 'N', 24: 'O', 25: 'P',26: 'Q', 27: 'R', 28: 'S', 29: 'T', 30: 'U',31: 'V', 32: 'W',33: 'X',
 34: 'Y',35: 'Z',36: 'a',37: 'b',38: 'c',39: 'd',40: 'e',41: 'f',42: 'g',43: 'h',44: 'i',45: 'j',46: 'k',47: 'l',48: 'm', 49: 'n',50: 'o',51: 'p', 52: 'q',
 53: 'r', 54: 's', 55: 't', 56: 'u', 57: 'v', 58: 'w', 59: 'x', 60: 'y',
 61: 'z'}

In [14]:
def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result
co=-1
STRING=''
while True:
    co+=1
    
    image = Image.open('dump/'+str(co)+'.png')


    print(co)
    im_new = add_margin(image, 10, 10,10, 10, (255))
    resized_image = im_new.resize((32,32))

    import numpy as np
    a=np.asarray(resized_image)/255

    hehe=labs[np.argmax(model.predict([a.tolist()]))]
    #print(hehe)
    #os.remove('dump/'+str(co)+'.png')
    #image.save("imgo/"+str(co)+'_'+str(hehe)+ ".png")

    #Image.fromarray((a*255).astype('uint8'), mode='L').save("imgi/"+str(co)+'_'+str(hehe)+ ".png")

    #model.predict([a.tolist()])
    STRING+=str(hehe)


0
1
2
3
4
5
6
7


FileNotFoundError: [Errno 2] No such file or directory: 'dump/8.png'

In [15]:
STRING

'mdmWnJdm'

In [57]:
x2

16

In [46]:
y1

6

In [47]:
y2

17

In [48]:
a

array([[255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255, 255, 255],
       [255, 255, 255, 255, 255, 255, 255, 255, 255,   0,   0,   0,   0,
          0,   0, 255],
       [255, 255, 255, 255, 255, 255, 255,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [255, 255, 255, 255, 255, 255, 255,   0,   0,   0, 255, 255, 255,
        255,   0,   0],
       [255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
        255,   0,   0],
       [255, 255, 255, 255, 25

In [54]:
a[:,7]

array([255, 255, 255, 255, 255, 255, 255,   0,   0, 255, 255, 255,   0,
         0,   0,   0, 255], dtype=uint8)

In [18]:
a = io.imread(r'C:\Users\Sanket M\Downloads\onboarding\onboarding\ocr\dump\0.png')
   
#print(a)
for i in range(len(a)):
    for j in range(len(a[0])):
        if a[i][j] >200:
            a[i][j]=255
        else:
            a[i][j]=0
def flagCalc(i):
    flag = 0
    for j in range(len(i)):
        if  i[j]==0:
            flag=1

    return flag       
y1=0
y2=a.shape[0]
x1=0
x2=a.shape[1]




In [23]:
for i in range(len(a)-1):
    #print(flagCalc(a[i])) 
    if flagCalc(a[i])<flagCalc(a[i+1]):
        y2=a.shape[0]
        if (i+1)< y2:
            y1=i+1
            
    elif flagCalc(a[i])>flagCalc(a[i+1]):
        if (i-1)>y1:
            y2=i-1
for i in range(len(a[0,:])-1):
    #print(flagCalc(a[i])) 
    if flagCalc(a[:,i])<flagCalc(a[:,i+1]):
        if (i+1)< x2:
            x1=i+1
    elif flagCalc(a[:,i])>flagCalc(a[:,i+1]):
        if (i-1)>x1:
            x2=i-1   


In [24]:
y1

29

In [25]:
y2

114

In [ ]:
im = Image.fromarray(a[y1:y2,x1:x2])
print(y1,y2,x1,x2)
im.save(path)